# Introdução

A proposta desse notebook é realizar um experimento de análise de sentimentos sobre os reviews de filmes do Rotten Tomatoes. Seguirei a metodologia proposta no notebook anteriormente enviado, mas durante a execução optei por fazer pequenas modificações em relação as tecnologias.

## Preparação do dataset

A primeira etapa é fazer a leitura dos arquivos contendo os textos do review e carregar os dados em estruturas do numpy, tornando assim mais fácil trabalhar com outras ferramentas como os classificadores e os pré-processadores.

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

tokenizer = RegexpTokenizer(r'[a-zA-Z0-9_]{2,}') #Only alphanumeric with size >= 2
stemmer = PorterStemmer()
    
def apply_stemming(text):
        
    stemmings = []

    for word in tokenizer.tokenize(text):
        stemmings.append(stemmer.stem(word))

    return " ".join(stemmings)
    
def load_dataset():
    neg_file = open("rt-polarity/rt-polarity.neg")
    pos_file = open("rt-polarity/rt-polarity.pos")
    
    reviews = []
    stemmed_reviews = []
    labels = []
    
    def add_review(review):
        cleaned_review = review.rstrip()
        reviews.append(cleaned_review)
        stemmed_reviews.append(apply_stemming(cleaned_review))
                
    for neg_review in neg_file:
        add_review(neg_review)
        labels.append(0)
        
    for pos_review in pos_file:
        add_review(pos_review)
        labels.append(1)
    
    return np.array(reviews), np.array(stemmed_reviews), np.array(labels)

reviews, stemmed_reviews, labels = load_dataset()

Um detalhe importante é que o stemming já é realizado na etapa leitura, deixando o dataset com stemming aplicado pré-processado. Optei for fazer dessa forma, pois o processo de stemming é pesado e isso facilitou o desenvolvimento das demais etapas do experimento.

### Separação em folds

Para podermos fazer um teste-t pareado, optei por separar os folds do conjunto no início. Adicionando uma coluna com os folds ao dataset, é possível re-utilizar essa divisão em vários experimentos.

In [2]:
from sklearn.model_selection import StratifiedKFold

def prepare_datase(reviews, stemmed_reviews, labels):
    
    skf = StratifiedKFold(n_splits=10,shuffle=True)
    skf.get_n_splits(reviews, labels)

    folds = np.zeros(len(labels), dtype=np.int)
    num_fold = 1

    for train_set, test_set in skf.split(reviews, labels):
    
        for index in test_set:
            folds[index] = num_fold
    
        num_fold += 1
        
    return np.vstack((reviews, stemmed_reviews, labels, folds)).T
    
dataset = prepare_datase(reviews, stemmed_reviews, labels)

Com os folds separados, um método simples permite obter conjuntos de treino e teste para validação cruzada.

In [3]:
def split_data(dataset, fold):

    validation = dataset[dataset[:,3] == fold]
    training = dataset[dataset[:,3] != fold]
    
    return (validation, training)

### Definição do vocabulário

Como optei for fazer a validação cruzada manualamente, é interessante termos definido o vocabulário do dataset. Lembrando que é necessário obter vocabulário da versão com stemming e sem stemming, já que a ideia do stemming é justamente criar um vocabulário reduzido.

A última etapa é construir os vetores de características para entrada nos classificadores.

In [9]:
def get_vocabulary(raw_text):

    vocabulary = set()
    
    for review in raw_text:

        for word in tokenizer.tokenize(review):
            vocabulary |= set(tokenizer.tokenize(review))
    
    return vocabulary

vocabulary_original = get_vocabulary(dataset[:,0])
vocabulary_stemmed = get_vocabulary(dataset[:,1])

### Transformação de textos em vetor de características

O scikit-learn oferece ferramentas para transformação de texto em representação do tipo bag-of-wods, no método abaixo as ferramentas do scikit-learn foram encapsuladas para gerarem os vetores de características de acordo com os experimentos propostos.

O método aceita as seguintes opções:

* `stemming`: booleano indicando se deve usar vocabulário com ou sem aplicação de stemming
* `representation_type`: indica se será utilizado a contagem de palavras (`"count"`), presença de palavras (`"binary"`) ou representação TF-IDF (`"tf-idf"`).
* `stop_words`: identifica se as stop words serão removidos na representação ou não.
* `n_gram` = quantidade de n-gramas utilizado na representação.


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def to_bag_of_words(raw_text, stemming, representation_type, stop_words, n_gram=1):
    
    use_stop_words = "english" if stop_words else None
    vocabulary = vocabulary_stemmed if stemming else vocabulary_original
    
    if representation_type == 'binary':
        vectorizer = CountVectorizer(ngram_range=(1, n_gram), binary=True, analyzer='word', vocabulary=vocabulary, stop_words=use_stop_words)
    elif representation_type == 'count':
        vectorizer = CountVectorizer(ngram_range=(1, n_gram), binary=False, analyzer='word', vocabulary=vocabulary, stop_words=use_stop_words)
    elif representation_type == 'tf-idf':
        vectorizer = TfidfVectorizer(ngram_range=(1, n_gram), analyzer='word', vocabulary=vocabulary, stop_words=use_stop_words)
    
    return vectorizer.fit_transform(raw_text)


## Execução dos algoritmos

Para validar os algoritmos, fiz um "grid-search" considerando as alterações no tipo de representação e pré-processamentos. Alterações nos parâmetros dos algoritmos, como por exemplo o parâmetro $ C $ do SVM poderiam ser incluídos nessa etapa.

Ao final de cada processamento, os resultados de cada fold são armazenados em uma mapa.

In [21]:
from sklearn import svm
from sklearn import naive_bayes
from sklearn import metrics

classifiers = {"naive_bayes": {"multinomial": naive_bayes.MultinomialNB(),
                               "bernoulli": naive_bayes.BernoulliNB(),
                               "gaussian": naive_bayes.GaussianNB()},
               "SVM": svm.LinearSVC()}
results = {}

teste = 0

for name_classifier, classifier in classifiers.items():
    
    name_key = name_classifier + "_"
    
    for stemming in [False, True]:

        stemming_key = name_key + str(stemming) + "_"
        
        for stop_words in [False, True]:

            stop_key = stemming_key + str(stop_words) + "_"

            for representation in ['binary', 'count', 'tf-idf']:

                representation_key = stop_key + representation + "_"

                for ngrams in [1, 2, 3]:

                    ngram_key = representation_key + str(ngrams)
                    results[ngram_key] = []

                    for fold in range(1,11):

                        validation, training = split_data(dataset, str(fold))
                        text_column = 1 if stemming else 0
                        
                        X_validation = to_bag_of_words(validation[:,text_column], stemming, representation, stop_words, ngrams)
                        X_training = to_bag_of_words(training[:,text_column], stemming, representation, stop_words, ngrams)

                        if name_classifier == "SVM":
                            
                            classifier.fit(X_training, training[:,2])
                            predictions = classifier.predict(X_validation)
                            
                        elif name_classifier == "naive_bayes":
                            
                            if representation == 'binary':
                                nb_classifier = classifier["bernoulli"]
                                nb_classifier.fit(X_training, training[:,2])
                            elif representation == 'count':
                                nb_classifier = classifier["multinomial"]
                                nb_classifier.fit(X_training, training[:,2])
                            elif representation == 'tf-idf':
                                nb_classifier = classifier["multinomial"]
                                nb_classifier.fit(X_training, training[:,2])
                            
                            
                            predictions = nb_classifier.predict(X_validation)
                            
                        accuracy = metrics.accuracy_score(validation[:,2], predictions)
                        results[ngram_key].append(accuracy)

                        print(f"Classifier: {name_classifier} Representation: {representation} n-grams: {ngrams} fold: {fold} stopwords: {stop_words} stemming: {stemming}: {accuracy}")

Classifier: naive_bayes Representation: binary n-grams: 1 fold: 1 stopwords: False stemming: False: 0.795880149812734
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 2 stopwords: False stemming: False: 0.776735459662289
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 3 stopwords: False stemming: False: 0.776735459662289
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 4 stopwords: False stemming: False: 0.7889305816135085
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 5 stopwords: False stemming: False: 0.7636022514071295
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 6 stopwords: False stemming: False: 0.7795497185741088
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 7 stopwords: False stemming: False: 0.7917448405253283
Classifier: naive_bayes Representation: binary n-grams: 1 fold: 8 stopwords: False stemming: False: 0.7664165103189493
Classifier: naive_bayes Representation: binary n-gr

Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 1 stopwords: False stemming: False: 0.7921348314606742
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 2 stopwords: False stemming: False: 0.7804878048780488
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 3 stopwords: False stemming: False: 0.776735459662289
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 4 stopwords: False stemming: False: 0.7786116322701688
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 5 stopwords: False stemming: False: 0.773921200750469
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 6 stopwords: False stemming: False: 0.7851782363977486
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 7 stopwords: False stemming: False: 0.7889305816135085
Classifier: naive_bayes Representation: tf-idf n-grams: 2 fold: 8 stopwords: False stemming: False: 0.773921200750469
Classifier: naive_bayes Representation: tf-idf n-gr

Classifier: naive_bayes Representation: count n-grams: 3 fold: 1 stopwords: True stemming: False: 0.7883895131086143
Classifier: naive_bayes Representation: count n-grams: 3 fold: 2 stopwords: True stemming: False: 0.7729831144465291
Classifier: naive_bayes Representation: count n-grams: 3 fold: 3 stopwords: True stemming: False: 0.7589118198874296
Classifier: naive_bayes Representation: count n-grams: 3 fold: 4 stopwords: True stemming: False: 0.7589118198874296
Classifier: naive_bayes Representation: count n-grams: 3 fold: 5 stopwords: True stemming: False: 0.7467166979362101
Classifier: naive_bayes Representation: count n-grams: 3 fold: 6 stopwords: True stemming: False: 0.7682926829268293
Classifier: naive_bayes Representation: count n-grams: 3 fold: 7 stopwords: True stemming: False: 0.7682926829268293
Classifier: naive_bayes Representation: count n-grams: 3 fold: 8 stopwords: True stemming: False: 0.7636022514071295
Classifier: naive_bayes Representation: count n-grams: 3 fold: 9

Classifier: naive_bayes Representation: count n-grams: 1 fold: 1 stopwords: False stemming: True: 0.8071161048689138
Classifier: naive_bayes Representation: count n-grams: 1 fold: 2 stopwords: False stemming: True: 0.7711069418386491
Classifier: naive_bayes Representation: count n-grams: 1 fold: 3 stopwords: False stemming: True: 0.7776735459662288
Classifier: naive_bayes Representation: count n-grams: 1 fold: 4 stopwords: False stemming: True: 0.7823639774859287
Classifier: naive_bayes Representation: count n-grams: 1 fold: 5 stopwords: False stemming: True: 0.7729831144465291
Classifier: naive_bayes Representation: count n-grams: 1 fold: 6 stopwords: False stemming: True: 0.7842401500938087
Classifier: naive_bayes Representation: count n-grams: 1 fold: 7 stopwords: False stemming: True: 0.7814258911819888
Classifier: naive_bayes Representation: count n-grams: 1 fold: 8 stopwords: False stemming: True: 0.7598499061913696
Classifier: naive_bayes Representation: count n-grams: 1 fold: 9

Classifier: naive_bayes Representation: binary n-grams: 2 fold: 1 stopwords: True stemming: True: 0.7893258426966292
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 2 stopwords: True stemming: True: 0.7617260787992496
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 3 stopwords: True stemming: True: 0.7532833020637899
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 4 stopwords: True stemming: True: 0.7654784240150094
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 5 stopwords: True stemming: True: 0.7607879924953096
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 6 stopwords: True stemming: True: 0.7851782363977486
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 7 stopwords: True stemming: True: 0.775797373358349
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 8 stopwords: True stemming: True: 0.7626641651031895
Classifier: naive_bayes Representation: binary n-grams: 2 fold: 9

Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 2 stopwords: True stemming: True: 0.7654784240150094
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 3 stopwords: True stemming: True: 0.7579737335834896
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 4 stopwords: True stemming: True: 0.7701688555347092
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 5 stopwords: True stemming: True: 0.7570356472795498
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 6 stopwords: True stemming: True: 0.7776735459662288
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 7 stopwords: True stemming: True: 0.774859287054409
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 8 stopwords: True stemming: True: 0.7664165103189493
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 9 stopwords: True stemming: True: 0.7654784240150094
Classifier: naive_bayes Representation: tf-idf n-grams: 3 fold: 1

Classifier: SVM Representation: tf-idf n-grams: 1 fold: 6 stopwords: False stemming: False: 0.7673545966228893
Classifier: SVM Representation: tf-idf n-grams: 1 fold: 7 stopwords: False stemming: False: 0.7598499061913696
Classifier: SVM Representation: tf-idf n-grams: 1 fold: 8 stopwords: False stemming: False: 0.7485928705440901
Classifier: SVM Representation: tf-idf n-grams: 1 fold: 9 stopwords: False stemming: False: 0.7729831144465291
Classifier: SVM Representation: tf-idf n-grams: 1 fold: 10 stopwords: False stemming: False: 0.7664165103189493
Classifier: SVM Representation: tf-idf n-grams: 2 fold: 1 stopwords: False stemming: False: 0.7743445692883895
Classifier: SVM Representation: tf-idf n-grams: 2 fold: 2 stopwords: False stemming: False: 0.7673545966228893
Classifier: SVM Representation: tf-idf n-grams: 2 fold: 3 stopwords: False stemming: False: 0.7861163227016885
Classifier: SVM Representation: tf-idf n-grams: 2 fold: 4 stopwords: False stemming: False: 0.7870544090056285


Classifier: SVM Representation: count n-grams: 3 fold: 1 stopwords: True stemming: False: 0.75187265917603
Classifier: SVM Representation: count n-grams: 3 fold: 2 stopwords: True stemming: False: 0.7429643527204502
Classifier: SVM Representation: count n-grams: 3 fold: 3 stopwords: True stemming: False: 0.7166979362101313
Classifier: SVM Representation: count n-grams: 3 fold: 4 stopwords: True stemming: False: 0.7326454033771107
Classifier: SVM Representation: count n-grams: 3 fold: 5 stopwords: True stemming: False: 0.7363977485928705
Classifier: SVM Representation: count n-grams: 3 fold: 6 stopwords: True stemming: False: 0.7410881801125704
Classifier: SVM Representation: count n-grams: 3 fold: 7 stopwords: True stemming: False: 0.7382739212007504
Classifier: SVM Representation: count n-grams: 3 fold: 8 stopwords: True stemming: False: 0.7382739212007504
Classifier: SVM Representation: count n-grams: 3 fold: 9 stopwords: True stemming: False: 0.7401500938086304
Classifier: SVM Repre

Classifier: SVM Representation: count n-grams: 1 fold: 6 stopwords: False stemming: True: 0.7392120075046904
Classifier: SVM Representation: count n-grams: 1 fold: 7 stopwords: False stemming: True: 0.7448405253283302
Classifier: SVM Representation: count n-grams: 1 fold: 8 stopwords: False stemming: True: 0.7457786116322702
Classifier: SVM Representation: count n-grams: 1 fold: 9 stopwords: False stemming: True: 0.7354596622889306
Classifier: SVM Representation: count n-grams: 1 fold: 10 stopwords: False stemming: True: 0.7298311444652908
Classifier: SVM Representation: count n-grams: 2 fold: 1 stopwords: False stemming: True: 0.7631086142322098
Classifier: SVM Representation: count n-grams: 2 fold: 2 stopwords: False stemming: True: 0.7429643527204502
Classifier: SVM Representation: count n-grams: 2 fold: 3 stopwords: False stemming: True: 0.7279549718574109
Classifier: SVM Representation: count n-grams: 2 fold: 4 stopwords: False stemming: True: 0.7570356472795498
Classifier: SVM Re

Classifier: SVM Representation: binary n-grams: 3 fold: 1 stopwords: True stemming: True: 0.7387640449438202
Classifier: SVM Representation: binary n-grams: 3 fold: 2 stopwords: True stemming: True: 0.7204502814258912
Classifier: SVM Representation: binary n-grams: 3 fold: 3 stopwords: True stemming: True: 0.7063789868667918
Classifier: SVM Representation: binary n-grams: 3 fold: 4 stopwords: True stemming: True: 0.7345215759849906
Classifier: SVM Representation: binary n-grams: 3 fold: 5 stopwords: True stemming: True: 0.723264540337711
Classifier: SVM Representation: binary n-grams: 3 fold: 6 stopwords: True stemming: True: 0.7195121951219512
Classifier: SVM Representation: binary n-grams: 3 fold: 7 stopwords: True stemming: True: 0.7185741088180112
Classifier: SVM Representation: binary n-grams: 3 fold: 8 stopwords: True stemming: True: 0.7373358348968105
Classifier: SVM Representation: binary n-grams: 3 fold: 9 stopwords: True stemming: True: 0.7279549718574109
Classifier: SVM Repr

Algo interessante a observar é que o Naive Bayes multinomial funciona com  representação TF-IDF, inicialmente testei com [Naive Bayes Gaussiano](http://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) pelo TF-IDF ser uma representação de dados contínua, mas obtive péssimos resultados. Aparentemente, é implementada uma solução que adapta o [algoritmo multinomial ao TF-IDF](http://www.cs.waikato.ac.nz/ml/publications/2004/kibriya_et_al_cr.pdf).

## Validação dos resultados

Na metodologia, apontei que faria a validação dos resultados usando o R, msa para manter tudo em apenas um notebook achei mais simples seguir com Pyhton usando o scipy e pandas para as validações necessárias.

Abaixo, criei um DataFrame montando um cartesiano entre as execuções, contabilizando a média de acurácia de cada algoritmo e calculando o teste-t para verificar se as médias são diferentes.

In [22]:
from scipy import stats
import pandas as pd

def tabulate_results(results):
    
    processed_pairs = set()
    
    col_combination_1 = []
    col_combination_2 = []
    col_accuracy_1 = []
    col_accuracy_2 = []
    col_p_test = []

    for combination_1, accuracy_1 in results.items():
        for combination_2, accuracy_2 in results.items():
    
            if combination_1 == combination_2 or combination_2 + "|" + combination_1 in processed_pairs:
                continue
            
            accuracy_avg_1 = sum(accuracy_1)/len(accuracy_1)
            accuracy_avg_2 = sum(accuracy_2)/len(accuracy_2)
            
            processed_pairs.add(combination_2 + "|" + combination_1)
            
            col_combination_1.append(combination_1)
            col_combination_2.append(combination_2)
            col_accuracy_1.append(accuracy_avg_1)
            col_accuracy_2.append(accuracy_avg_2)
            col_p_test.append(stats.ttest_rel(accuracy_1,accuracy_2))
    
    return pd.DataFrame({"combination_1": col_combination_1,
                         "combination_2": col_combination_2,
                         "accuracy_1": col_accuracy_1,
                         "accuracy_2": col_accuracy_2,
                         "p_test": col_p_test})

tabulated_results = tabulate_results(results)

### Encontrando os melhores algoritmos

Para encontrar as melhores combinações de algoirtmos, ordenei o data_frame criado peloa média de acurácia dos algoritmos. Dessa forma, os melhores algoritmos ficarão no topo.

À primeira vista, podemos avaliar que o Naive Bayes obteve bons resultados, com variação desprezível entre os diferentes tipos de representação e pré-processamento. Além disso, o uso de $ n-gram > 1 $ não pareceu trazer ganhos na classificação também.

In [23]:
tabulated_results.sort_values(['accuracy_1', 'accuracy_2'], ascending=[0, 0])

,accuracy_1,accuracy_2,combination_1,combination_2,p_test
432,0.781653,0.781653,naive_bayes_False_False_tf-idf_1,naive_bayes_False_False_tf-idf_2,"(nan, nan)"
433,0.781653,0.781653,naive_bayes_False_False_tf-idf_1,naive_bayes_False_False_tf-idf_3,"(nan, nan)"
503,0.781653,0.781653,naive_bayes_False_False_tf-idf_2,naive_bayes_False_False_tf-idf_1,"(nan, nan)"
504,0.781653,0.781653,naive_bayes_False_False_tf-idf_2,naive_bayes_False_False_tf-idf_3,"(nan, nan)"
574,0.781653,0.781653,naive_bayes_False_False_tf-idf_3,naive_bayes_False_False_tf-idf_1,"(nan, nan)"
575,0.781653,0.781653,naive_bayes_False_False_tf-idf_3,naive_bayes_False_False_tf-idf_2,"(nan, nan)"
449,0.781653,0.781274,naive_bayes_False_False_tf-idf_1,naive_bayes_True_False_tf-idf_1,"(0.152388667931, 0.882241989021)"
450,0.781653,0.781274,naive_bayes_False_False_tf-idf_1,naive_bayes_True_False_tf-idf_2,"(0.152388667931, 0.882241989021)"
451,0.781653,0.781274,naive_bayes_False_False_tf-idf_1,naive_bayes_True_False_tf-idf_3,"(0.152388667931, 0.882241989021)"
520,0.781653,0.781274,naive_bayes_False_False_tf-idf_2,naive_bayes_True_False_tf-idf_1,"(0.152388667931, 0.882241989021)"


Para confirmar se as variações do Naive Bayes não foram relevantes, vamos verificar a maior diferença de média entre diferente configurações do algoritmo. Nesse caso, vemos que a contagem tem um desempenho signigificativamente pior que usando representação TF-IDF, enquanto o uso de representação binária mantém o mesmo desempenho do TF-IDF.

In [31]:
tabulated_results[tabulated_results.combination_2.str.startswith('naive_bayes')].sort_values(['accuracy_1', 'accuracy_2'], ascending=[0, 1])


,accuracy_1,accuracy_2,combination_1,combination_2,p_test
437,0.781653,0.766456,naive_bayes_False_False_tf-idf_1,naive_bayes_False_True_count_1,"(6.80649592923, 7.84776011779e-05)"
438,0.781653,0.766456,naive_bayes_False_False_tf-idf_1,naive_bayes_False_True_count_2,"(6.80649592923, 7.84776011779e-05)"
439,0.781653,0.766456,naive_bayes_False_False_tf-idf_1,naive_bayes_False_True_count_3,"(6.80649592923, 7.84776011779e-05)"
508,0.781653,0.766456,naive_bayes_False_False_tf-idf_2,naive_bayes_False_True_count_1,"(6.80649592923, 7.84776011779e-05)"
509,0.781653,0.766456,naive_bayes_False_False_tf-idf_2,naive_bayes_False_True_count_2,"(6.80649592923, 7.84776011779e-05)"
510,0.781653,0.766456,naive_bayes_False_False_tf-idf_2,naive_bayes_False_True_count_3,"(6.80649592923, 7.84776011779e-05)"
579,0.781653,0.766456,naive_bayes_False_False_tf-idf_3,naive_bayes_False_True_count_1,"(6.80649592923, 7.84776011779e-05)"
580,0.781653,0.766456,naive_bayes_False_False_tf-idf_3,naive_bayes_False_True_count_2,"(6.80649592923, 7.84776011779e-05)"
581,0.781653,0.766456,naive_bayes_False_False_tf-idf_3,naive_bayes_False_True_count_3,"(6.80649592923, 7.84776011779e-05)"
455,0.781653,0.767582,naive_bayes_False_False_tf-idf_1,naive_bayes_True_True_count_1,"(6.16812451128, 0.000165073579029)"


Em comparação ao SVM, podemos verificar que o Naive Bayes obteve melhor desempenho mesmo contra as melhores configurações utilizadas com SVM, que teve desempenho mais próximo ao Naive Bayes utilizando contagem de palavras.

Vale observar que é interessante experimentar outras configurações no algoritmo do SVM, como alterações nos fatores de penalidade e regularização, assim mesmo como utilizar outros tipo de Kernel.

In [32]:
tabulated_results[tabulated_results.combination_1.str.startswith('SVM')].sort_values(['accuracy_1', 'accuracy_2'], ascending=[0, 0])

,accuracy_1,accuracy_2,combination_1,combination_2,p_test
2988,0.770211,0.781653,SVM_False_False_tf-idf_1,naive_bayes_False_False_tf-idf_1,"(-2.77613027251, 0.0215365744699)"
2989,0.770211,0.781653,SVM_False_False_tf-idf_1,naive_bayes_False_False_tf-idf_2,"(-2.77613027251, 0.0215365744699)"
2990,0.770211,0.781653,SVM_False_False_tf-idf_1,naive_bayes_False_False_tf-idf_3,"(-2.77613027251, 0.0215365744699)"
3059,0.770211,0.781653,SVM_False_False_tf-idf_2,naive_bayes_False_False_tf-idf_1,"(-2.77613027251, 0.0215365744699)"
3060,0.770211,0.781653,SVM_False_False_tf-idf_2,naive_bayes_False_False_tf-idf_2,"(-2.77613027251, 0.0215365744699)"
3061,0.770211,0.781653,SVM_False_False_tf-idf_2,naive_bayes_False_False_tf-idf_3,"(-2.77613027251, 0.0215365744699)"
3130,0.770211,0.781653,SVM_False_False_tf-idf_3,naive_bayes_False_False_tf-idf_1,"(-2.77613027251, 0.0215365744699)"
3131,0.770211,0.781653,SVM_False_False_tf-idf_3,naive_bayes_False_False_tf-idf_2,"(-2.77613027251, 0.0215365744699)"
3132,0.770211,0.781653,SVM_False_False_tf-idf_3,naive_bayes_False_False_tf-idf_3,"(-2.77613027251, 0.0215365744699)"
3006,0.770211,0.781274,SVM_False_False_tf-idf_1,naive_bayes_True_False_tf-idf_1,"(-2.64847613839, 0.0265424131144)"


Comparando o resultado das diferenças configurações do SVM, é interessante notar que a representação teve um impacto considerável na performance do algoritmo, sendo o TF-IDF novamente a melhor opção para representação.

In [34]:
svm_results =  tabulated_results[tabulated_results.combination_1.str.startswith('SVM')]
svm_results_only = svm_results[svm_results.combination_2.str.startswith('SVM')]
svm_results_only.sort_values(['accuracy_1', 'accuracy_2'], ascending=[0, 1])

,accuracy_1,accuracy_2,combination_1,combination_2,p_test
3044,0.770211,0.724345,SVM_False_False_tf-idf_1,SVM_True_True_binary_1,"(8.62449229291, 1.20830590488e-05)"
3045,0.770211,0.724345,SVM_False_False_tf-idf_1,SVM_True_True_binary_2,"(8.62449229291, 1.20830590488e-05)"
3046,0.770211,0.724345,SVM_False_False_tf-idf_1,SVM_True_True_binary_3,"(8.62449229291, 1.20830590488e-05)"
3115,0.770211,0.724345,SVM_False_False_tf-idf_2,SVM_True_True_binary_1,"(8.62449229291, 1.20830590488e-05)"
3116,0.770211,0.724345,SVM_False_False_tf-idf_2,SVM_True_True_binary_2,"(8.62449229291, 1.20830590488e-05)"
3117,0.770211,0.724345,SVM_False_False_tf-idf_2,SVM_True_True_binary_3,"(8.62449229291, 1.20830590488e-05)"
3186,0.770211,0.724345,SVM_False_False_tf-idf_3,SVM_True_True_binary_1,"(8.62449229291, 1.20830590488e-05)"
3187,0.770211,0.724345,SVM_False_False_tf-idf_3,SVM_True_True_binary_2,"(8.62449229291, 1.20830590488e-05)"
3188,0.770211,0.724345,SVM_False_False_tf-idf_3,SVM_True_True_binary_3,"(8.62449229291, 1.20830590488e-05)"
3047,0.770211,0.725377,SVM_False_False_tf-idf_1,SVM_True_True_count_1,"(8.09509494022, 2.01382270753e-05)"


## Conclusão

Os algoritmos tiveram performances similares, mas o Naive Bayes chegou a resultados melhores que o SVM.

É interessante observar que no SVM o tipo de representação teve grande impacto, mas as demais variações não parecem ter sido muito relevantes à primeira vista nos resultados de classificação: stemming, remoção de stopwords e uso de n-gramas. Para próximos experimentos em problemas similares, talvez seja mais vantajoso trocar o foco: ao invés de investir em pré-processamento, focar em experimentar mais classificadores e alterar as configurações do mesmo.

Infelizmente, faltou a implementação da solução utilizando rede neural. Talvez, seja possível obter resultados consideravelmente superiores já que estamos falando de uma abordagem completamente diferente para o problema.

Para os próximos passos, seria interessante uma análise mais cuidadosa dos dados obtidos, explorar melhor o SVM e avaliar uma solução de rede neural.